In [3]:
!pip install azure-identity
!pip install azure-keyvault-secrets

  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)


In [5]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# Replace with your Key Vault URL
key_vault_url = "https://azuredevassociate.vault.azure.net/"  

# Use DefaultAzureCredential to authenticate
credential = DefaultAzureCredential()

# Create a SecretClient instance
client = SecretClient(vault_url=key_vault_url, credential=credential)

In [9]:
# Show all the secrets in the Key Vault
secrets = client.list_properties_of_secrets()
for secret in secrets:
    print(secret.name)

HttpResponseError: (Forbidden) Caller is not authorized to perform action on resource.
If role assignments, deny assignments or role definitions were changed recently, please observe propagation time.
Caller: appid=04b07795-8ddb-461a-bbee-02f9e1bf7b46;oid=294baa14-acc3-41bb-b015-6ccae670d22a;iss=https://sts.windows.net/db129cb8-5bda-4db1-8f8e-c6a4228987c2/
Action: 'Microsoft.KeyVault/vaults/secrets/readMetadata/action'
Resource: '/subscriptions/66ff019d-3298-4b17-b525-bc7b89191a6c/resourcegroups/rg-prod-azure-developer-associate-swedencentral-001/providers/microsoft.keyvault/vaults/azuredevassociate'
Assignment: (not found)
DenyAssignmentId: null
DecisionReason: null 
Vault: azuredevassociate;location=swedencentral

Code: Forbidden
Message: Caller is not authorized to perform action on resource.
If role assignments, deny assignments or role definitions were changed recently, please observe propagation time.
Caller: appid=04b07795-8ddb-461a-bbee-02f9e1bf7b46;oid=294baa14-acc3

In [ ]:
from azure.storage.blob import BlobClient

# When deployed to an azure host, the default azure credential will authenticate the specified user assigned managed identity.
user_assigned_client_id = "<your managed identity client Id>"
credential = DefaultAzureCredential(managed_identity_client_id=user_assigned_client_id)

blob_client = BlobClient(account_url="https://myaccount.blob.core.windows.net", container_name="mycontainer", blob_name="myblob", credential=credential)